In [1]:
import sys
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import skimage as ski
import PIL as PIL
import time

from skimage import measure
from mpl_toolkits.mplot3d import Axes3D 
from PIL import Image

np.set_printoptions(threshold = 1000)

Task 2: Image derivatives with anisotropic voxel dimensions

Implement a class Image3D, which should handle 3D medical images with different voxel dimensions, image sizes and data types.

In [6]:
def __init__(self, image, vox_x, vox_y, vox_z):
    pass